### Feature Engineering  
In diesem Notebook wird die **raw** CSV "data" um weitere Spalten ergänzt, die möglicherweise weitere Predictors sein könnten.

#### Imports

In [1]:
import pandas as pd


import importlib
import utils
# Cache-Probleme umgehen
importlib.reload(utils)
from utils import left
from utils import right
from utils import get_season
from utils import rearrange_col

#### Daten laden

In [32]:
path_data = 'https://raw.githubusercontent.com/mm391-030401/project/refs/heads/main/data/raw/'
file_data = 'data.csv'

data = pd.read_csv(path_data + file_data, sep=',', encoding='utf-8')

#### Überblick über raw-data

In [33]:
data.head()

,MONATE_SEIT_EINFUEHRUNG_PROGRAMM_KOHORTE,MONAT,KOHORTE,ERSTER_MONAT_KOHORTE_FG,MONATE_SEIT_EXISTENZ_KOHORTE,KOHORTENGROESSE_INDEXIERT,IDENTIFIZIERTE_KUNDEN_INDEXIERT,RABATT_INDEXIERT
0,-2,201408,201408,1,0,0.41,0.41,2.54
1,-2,201409,201408,0,1,0.41,0.40,7.14
2,-2,201410,201408,0,2,0.41,0.39,9.28
3,-2,201411,201408,0,3,0.41,0.38,3.22
4,-2,201412,201408,0,4,0.41,0.38,7.15


In [34]:
data.columns

Index(['MONATE_SEIT_EINFUEHRUNG_PROGRAMM_KOHORTE', 'MONAT', 'KOHORTE',
       'ERSTER_MONAT_KOHORTE_FG', 'MONATE_SEIT_EXISTENZ_KOHORTE',
       'KOHORTENGROESSE_INDEXIERT', 'IDENTIFIZIERTE_KUNDEN_INDEXIERT',
       'RABATT_INDEXIERT'],
      dtype='object')

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703 entries, 0 to 702
Data columns (total 8 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   MONATE_SEIT_EINFUEHRUNG_PROGRAMM_KOHORTE  703 non-null    int64  
 1   MONAT                                     703 non-null    int64  
 2   KOHORTE                                   703 non-null    int64  
 3   ERSTER_MONAT_KOHORTE_FG                   703 non-null    int64  
 4   MONATE_SEIT_EXISTENZ_KOHORTE              703 non-null    int64  
 5   KOHORTENGROESSE_INDEXIERT                 703 non-null    float64
 6   IDENTIFIZIERTE_KUNDEN_INDEXIERT           703 non-null    float64
 7   RABATT_INDEXIERT                          703 non-null    float64
dtypes: float64(3), int64(5)
memory usage: 44.1 KB


#### Feature Engineering  
Nach Betrachtung der vorhandenen Spalten werden weitere Zeitvariablen als sinnvoll erachtet. Aktuell werden der Monat sowie die Kohorte im Format JJJJMM angegeben.  
Damit das Modell Saisonalitäten zur Jahreszeit, zum Monat sowie einen allgemeinen Trend bei zunehmender Jahreszahl erkennen kann, werden die Spalten `Monat` und `Kohorte` in weitere Zeitvariblen zerlegt.

Um die Werte in den Spalten zerlegen zu können, werden zwei Funktionen verwendet.  
- `left()` gibt den linken Part eines Werts aus. In der Funktion kann definiert werden, wie viele Zeichen des linken Parts ausgegeben werden.  
- `right()` gibt den rechten Part eines Werts aus. In der Funktion kann definiert werden, wie viele Zeichen des rechten Parts ausgegeben werden.  

In [38]:
data['monat_jahr'] = data['MONAT'].astype(str).apply(lambda x: left(x)).astype(int)
data['monat_monat'] = data['MONAT'].astype(str).apply(lambda x: right(x)).astype(int)

data['kohorte_jahr'] = data['KOHORTE'].astype(str).apply(lambda x: left(x)).astype(int)
data['kohorte_monat'] = data['KOHORTE'].astype(str).apply(lambda x: right(x)).astype(int)

In [39]:
# Überprüfung, ob Funktionen richtig funktioniert haben
data[['MONAT', 'monat_jahr', 'monat_monat', 'KOHORTE', 'kohorte_jahr', 'kohorte_monat']].head()

,MONAT,monat_jahr,monat_monat,KOHORTE,kohorte_jahr,kohorte_monat
0,201408,2014,8,201408,2014,8
1,201409,2014,9,201408,2014,8
2,201410,2014,10,201408,2014,8
3,201411,2014,11,201408,2014,8
4,201412,2014,12,201408,2014,8


Um die Jahreszeiten zu bestimmen wird eine einfache Aufteilung der Monate in Jahreszeiten angenommen (z.B. Dezemeber bis Februar entspricht Winter).

In [41]:
data['monat_jahreszeit'] = data['monat_monat'].apply(lambda x: get_season(x))

data['kohorte_jahreszeit'] = data['kohorte_monat'].apply(lambda x: get_season(x))

In [42]:
# Überprüfung, ob Funktionen richtig funktioniert haben
data[['monat_monat', 'monat_jahreszeit', 'kohorte_monat', 'kohorte_jahreszeit']].head()

,monat_monat,monat_jahreszeit,kohorte_monat,kohorte_jahreszeit
0,8,Sommer,8,Sommer
1,9,Herbst,8,Sommer
2,10,Herbst,8,Sommer
3,11,Herbst,8,Sommer
4,12,Winter,8,Sommer


Die neu generieten Spalten stehen nun am Ende des Dataframes. Um dies übersichtlich zu gestalten, werden die Spalten entsprechend umgeordnet.

In [44]:
data = rearrange_col(data, 'MONAT', ['monat_jahr', 'monat_monat', 'monat_jahreszeit'])
data = rearrange_col(data, 'KOHORTE', ['kohorte_jahr', 'kohorte_monat', 'kohorte_jahreszeit'])

Aus der Kohortengröße und den identifizierten Kunden wird die Retentionrate jeder Kohorte je Monat berechnet.

In [45]:
data['retentionrate'] = data['IDENTIFIZIERTE_KUNDEN_INDEXIERT'] / data['KOHORTENGROESSE_INDEXIERT'] * 100

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703 entries, 0 to 702
Data columns (total 15 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   MONATE_SEIT_EINFUEHRUNG_PROGRAMM_KOHORTE  703 non-null    int64  
 1   MONAT                                     703 non-null    int64  
 2   monat_jahr                                703 non-null    int64  
 3   monat_monat                               703 non-null    int64  
 4   monat_jahreszeit                          703 non-null    object 
 5   KOHORTE                                   703 non-null    int64  
 6   kohorte_jahr                              703 non-null    int64  
 7   kohorte_monat                             703 non-null    int64  
 8   kohorte_jahreszeit                        703 non-null    object 
 9   ERSTER_MONAT_KOHORTE_FG                   703 non-null    int64  
 10  MONATE_SEIT_EXISTENZ_KOHORTE          

In [47]:
data.head()

,MONATE_SEIT_EINFUEHRUNG_PROGRAMM_KOHORTE,MONAT,monat_jahr,monat_monat,monat_jahreszeit,KOHORTE,kohorte_jahr,kohorte_monat,kohorte_jahreszeit,ERSTER_MONAT_KOHORTE_FG,MONATE_SEIT_EXISTENZ_KOHORTE,KOHORTENGROESSE_INDEXIERT,IDENTIFIZIERTE_KUNDEN_INDEXIERT,RABATT_INDEXIERT,retentionrate
0,-2,201408,2014,8,Sommer,201408,2014,8,Sommer,1,0,0.41,0.41,2.54,100.000000
1,-2,201409,2014,9,Herbst,201408,2014,8,Sommer,0,1,0.41,0.40,7.14,97.560976
2,-2,201410,2014,10,Herbst,201408,2014,8,Sommer,0,2,0.41,0.39,9.28,95.121951
3,-2,201411,2014,11,Herbst,201408,2014,8,Sommer,0,3,0.41,0.38,3.22,92.682927
4,-2,201412,2014,12,Winter,201408,2014,8,Sommer,0,4,0.41,0.38,7.15,92.682927


#### Daten ablegen
Nun wird die neue Datengrundlage in den Ordner interim abgelegt

In [53]:
path_data_new = 'https://raw.githubusercontent.com/mm391-030401/project/refs/heads/main/data/interim/'
file_data_new = 'data_added_features.csv'

data.to_csv(path_data_new + file_data_new, index = False)